<a href="https://colab.research.google.com/github/bundickm/DS-Unit-1-Sprint-2-Data-Wrangling/blob/master/module4-make-features/LS_DS_124_Make_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

_Lambda School Data Science_

# Make features

Objectives
-  understand the purpose of feature engineering
-  work with strings in pandas
- work with dates and times in pandas

Links
- [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)
- Python Data Science Handbook
  - [Chapter 3.10](https://jakevdp.github.io/PythonDataScienceHandbook/03.10-working-with-strings.html), Vectorized String Operations
  - [Chapter 3.11](https://jakevdp.github.io/PythonDataScienceHandbook/03.11-working-with-time-series.html), Working with Time Series

## Get LendingClub data

[Source](https://www.lendingclub.com/info/download-data.action)

In [1]:
!wget https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip

--2019-03-28 22:56:40--  https://resources.lendingclub.com/LoanStats_2018Q4.csv.zip
Resolving resources.lendingclub.com (resources.lendingclub.com)... 64.48.1.20
Connecting to resources.lendingclub.com (resources.lendingclub.com)|64.48.1.20|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘LoanStats_2018Q4.csv.zip’

LoanStats_2018Q4.cs     [               <=>  ]  21.29M   878KB/s    in 25s     

2019-03-28 22:57:05 (869 KB/s) - ‘LoanStats_2018Q4.csv.zip’ saved [22329081]



In [2]:
!unzip LoanStats_2018Q4.csv.zip

Archive:  LoanStats_2018Q4.csv.zip
  inflating: LoanStats_2018Q4.csv    


In [0]:
!head LoanStats_2018Q4.csv

In [0]:
!tail LoanStats_2018Q4.csv

"","","5600","5600","5600"," 36 months"," 13.56%","190.21","C","C1","","n/a","RENT","15600","Not Verified","Oct-2018","Current","n","","","credit_card","Credit card refinancing","836xx","ID","15.31","0","Aug-2012","0","","97","9","1","5996","34.5%","11","w","5083.61","5083.61","750.29","750.29","516.39","233.90","0.0","0.0","0.0","Feb-2019","190.21","Mar-2019","Feb-2019","0","","1","Individual","","","","0","0","5996","0","0","0","1","20","0","","0","2","3017","35","17400","1","0","0","3","750","4689","45.5","0","0","20","73","13","13","0","13","","20","","0","3","5","4","4","1","9","10","5","9","0","0","0","0","100","25","1","0","17400","5996","8600","0","","","","","","","","","","","","N","","","","","","","","","","","","","","","Cash","N","","","","","",""
"","","23000","23000","23000"," 36 months"," 15.02%","797.53","C","C3","Tax Consultant","10+ years","MORTGAGE","75000","Source Verified","Oct-2018","Charged Off","n","","","debt_consolidation","Debt consolidation","352xx","AL","

## Load LendingClub data

pandas documentation
- [`read_csv`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html)
- [`options.display`](https://pandas.pydata.org/pandas-docs/stable/options.html#available-options)

In [0]:
import pandas as pd
import numpy as np

In [0]:
df = pd.read_csv('LoanStats_2018Q4.csv', skiprows=1, skipfooter=2, engine='python')

In [0]:
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500

In [0]:
df.shape

(128412, 145)

In [0]:
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,NaN,NaN,2500,2500,2500.0,36 months,13.56%,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,109xx,NY,18.24,0,Apr-2001,1,NaN,45.0,9,1,4341,10.3%,34,w,2386.02,2386.02,167.02,167.02,113.98,53.04,0.0,0.0,0.0,Feb-2019,84.92,Mar-2019,Feb-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,0,16901,2,2,1,2,2.0,12560,69.0,2,7,2137,28.0,42000,1,11,2,9,1878.0,34360.0,5.9,0,0,140.0,212,1,1,0,1.0,NaN,2.0,NaN,0,2,5,3,3,16,7,18,5,9,0.0,0,0,3,100.0,0.0,1,0,60124,16901,36500,18124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,30000,30000,30000.0,60 months,18.94%,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,713xx,LA,26.52,0,Jun-1987,0,71.0,75.0,13,1,12315,24.2%,44,w,29387.75,29387.75,1507.11,1507.11,612.25,894.86,0.0,0.0,0.0,Feb-2019,777.23,Mar-2019,Feb-2019,0,NaN,1,Individual,NaN,NaN,NaN,0,1208,321915,4,4,2,3,3.0,87153,88.0,4,5,998,57.0,50800,2,15,2,10,24763.0,13761.0,8.3,0,0,163.0,378,4,3,3,4.0,NaN,4.0,NaN,0,2,4,4,9,27,8,14,4,13,0.0,0,0,6,95.0,0.0,1,0,372872,99468,15000,94072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,5000,5000,5000.0,36 months,17.97%,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,Dec-2018,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,490xx,MI,10.51,0,Apr-2011,0,NaN,NaN,8,0,4599,19.1%,13,w,4787.21,4787.21,353.89,353.89,212.79,141.10,0.0,0.0,0.0,Feb-2019,180.69,Mar-2019,Feb-2019,0,NaN,1,Individual,NaN,NaN,

In [0]:
df.isnull().sum().sort_values()

total_cu_tl                                        0
mo_sin_old_rev_tl_op                               0
delinq_amnt                                        0
chargeoff_within_12_mths                           0
acc_open_past_24mths                               0
inq_last_12m                                       0
inq_fi                                             0
total_rev_hi_lim                                   0
max_bal_bc                                         0
open_rv_24m                                        0
open_rv_12m                                        0
total_bal_il                                       0
debt_settlement_flag                               0
open_il_12m                                        0
open_act_il                                        0
open_acc_6m                                        0
tot_cur_bal                                        0
tot_coll_amt                                       0
acc_now_delinq                                

## Work with strings

For machine learning, we usually want to replace strings with numbers.

We can get info about which columns have a datatype of "object" (strings)

In [0]:
df.describe(exclude='number')

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,next_pymnt_d,last_credit_pull_d,application_type,verification_status_joint,sec_app_earliest_cr_line,hardship_flag,hardship_type,hardship_reason,hardship_status,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_loan_status,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date
count,128412,128412,128412,128412,107465,116708,128412,128412,128412,128412,128412,128412,128412,128412,128412,128412,128256,128412,128250,124941,128411,128412,14848,16782,128412,1,1,1,1,1,1,1,128412,128412,2,2,2
unique,2,46,7,35,43892,11,4,3,3,6,2,12,12,880,50,644,1074,2,5,3,7,2,3,573,2,1,1,1,1,1,1,1,2,2,1,1,1
top,36 months,13.56%,A,A4,Teacher,10+ years,MORTGAGE,Not Verified,Oct-2018,Current,n,debt_consolidation,Debt consolidation,112xx,CA,Aug-2006,0%,w,Feb-2019,Mar-2019,Feb-2019,Individual,Not Verified,Aug-2006,N,INTEREST ONLY-3 MONTHS DEFERRAL,UNEMPLOYMENT,ACTIVE,Feb-2019,Apr-2019,Feb-2019,Late (16-30 days),Cash,N,Feb-2019,ACTIVE,Feb-2019
freq,88179,6976,38011,9770,2090,38826,63490,58350,46305,123768,128411,70603,70603,1370,17879,1130,1132,114498,123797,124903,125061,111630,6360,155,128411,1,1,1,1,1,1,1,102516,128410,2,2,2


In [0]:
df.dtypes

id                                            float64
member_id                                     float64
loan_amnt                                       int64
funded_amnt                                     int64
funded_amnt_inv                               float64
term                                           object
int_rate                                       object
installment                                   float64
grade                                          object
sub_grade                                      object
emp_title                                      object
emp_length                                     object
home_ownership                                 object
annual_inc                                    float64
verification_status                            object
issue_d                                        object
loan_status                                    object
pymnt_plan                                     object
url                         

### Convert `int_rate`

In [0]:
'13.56%'.strip('%')

'13.56'

In [0]:
float('13.56%'.strip('%'))

13.56

In [0]:
df['int_rate'] = df['int_rate'].str.strip('%').astype(float)

Define a function to remove percent signs from strings and convert to floats

In [6]:
string = '13.56%'

def remove_percent(string):
  return float(string.strip('%'))

remove_percent(string)

13.56

Apply the function to the `int_rate` column

In [7]:
df['int_rate'] = df['int_rate'].apply(remove_percent)

AttributeError: ignored

### Clean `emp_title`

Look at top 20 titles

In [0]:
df['emp_title'].head(20)

0                            Chef
1                     Postmaster 
2                  Administrative
3                   IT Supervisor
4                        Mechanic
5                    Director COE
6                 Account Manager
7              Assistant Director
8             Legal Assistant III
9                             NaN
10                            NaN
11                     Consultant
12           Job Coach Supervisor
13         Quality Field Engineer
14                        Teller 
15           respritory therapist
16               Worship Director
17                     Processor 
18    Neonatal Nurse Practitioner
19            Stationary Engineer
Name: emp_title, dtype: object

In [0]:
df['emp_title'].value_counts().head(20)

Teacher                     2090
Manager                     1773
Registered Nurse             952
Driver                       924
RN                           726
Supervisor                   697
Sales                        580
Project Manager              526
General Manager              523
Office Manager               521
Owner                        420
Director                     402
Truck Driver                 387
Operations Manager           387
Nurse                        326
Engineer                     325
Sales Manager                304
manager                      301
Supervisor                   270
Administrative Assistant     269
Name: emp_title, dtype: int64

How often is `emp_title` null?

In [0]:
df['emp_title'].isnull().sum()

20947

Clean the title and handle missing values
* capitalize
* strip spaces
* replace NaN with Unknown

In [6]:
examples = ['owner', 'Supervisor ', ' Project Manager', np.nan]

def clean_title(x):
  if isinstance(x, str):
    return x.strip().title()
  else:
    return 'Unknown'

[clean_title(x) for x in examples]

['Owner', 'Supervisor', 'Project Manager', 'Unknown']

In [0]:
df['emp_title'] = df['emp_title'].apply(clean_title)

In [0]:
df['emp_title'].head()

In [0]:
df['emp_title'].value_counts().head(20)

Unknown               20947
Teacher                2557
Manager                2395
Registered Nurse       1418
Driver                 1258
Supervisor             1160
Truck Driver            920
Rn                      834
Office Manager          805
Sales                   803
General Manager         791
Project Manager         720
Owner                   625
Director                523
Operations Manager      518
Sales Manager           500
Police Officer          440
Nurse                   425
Technician              420
Engineer                412
Name: emp_title, dtype: int64

### Create `emp_title_manager`

pandas documentation: [`str.contains`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.contains.html)

In [0]:
df['emp_title_manager'] = df['emp_title'].str.contains('Manager')

In [0]:
df['emp_title_manager'].value_counts()

False    110530
True      17882
Name: emp_title_manager, dtype: int64

## Work with dates

pandas documentation
- [to_datetime](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.to_datetime.html)
- [Time/Date Components](https://pandas.pydata.org/pandas-docs/stable/timeseries.html#time-date-components) "You can access these properties via the `.dt` accessor"

In [0]:
df['issue_d'].head().values

array(['Dec-2018', 'Dec-2018', 'Dec-2018', 'Dec-2018', 'Dec-2018'],
      dtype=object)

In [0]:
df['issue_d'].describe()

count       128412
unique           3
top       Oct-2018
freq         46305
Name: issue_d, dtype: object

In [0]:
df['issue_d'] = pd.to_datetime(df['issue_d'], infer_datetime_format=True)

In [0]:
df['issue_d'].describe()

count                  128412
unique                      3
top       2018-10-01 00:00:00
freq                    46305
first     2018-10-01 00:00:00
last      2018-12-01 00:00:00
Name: issue_d, dtype: object

In [0]:
df['issue_month'] = df['issue_d'].dt.month.head()
df['issue_year'] = df['issue_d'].dt.year.head()

In [0]:
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], infer_datetime_format=True)

In [0]:
df['earliest_cr_line'].head()

0   2001-04-01
1   1987-06-01
2   2011-04-01
3   2006-02-01
4   2000-12-01
Name: earliest_cr_line, dtype: datetime64[ns]

In [0]:
df['dif_earliest_cr_to_issue'] = (df['issue_d'] - df['earliest_cr_line']).dt.days

In [0]:
[col for col in df if col.endswith('_d')]

['issue_d', 'last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']

In [0]:
for col in ['last_pymnt_d', 'next_pymnt_d', 'last_credit_pull_d']:
  df[col] = pd.to_datetime(df[col], infer_datetime_format=True)

In [0]:
df.dtypes

id                                                   float64
member_id                                            float64
loan_amnt                                              int64
funded_amnt                                            int64
funded_amnt_inv                                      float64
term                                                  object
int_rate                                             float64
installment                                          float64
grade                                                 object
sub_grade                                             object
emp_title                                             object
emp_length                                            object
home_ownership                                        object
annual_inc                                           float64
verification_status                                   object
issue_d                                       datetime64[ns]
loan_status             

# ASSIGNMENT
- Replicate the lesson code - *** Completed, see above***

- Convert the `term` column from string to integer.

In [0]:
#seeing whats in the column including NaN
df['term'].value_counts(dropna=False)

 36 months    88179
 60 months    40233
Name: term, dtype: int64

In [0]:
#convert term to int after stripping ' months'
df['term'] = df['term'].str.strip(' months').astype(int)

In [0]:
#confirming conversion
df['term'].describe(), df['term'].head()

(count    128412.000000
 mean         43.519484
 std          11.132203
 min          36.000000
 25%          36.000000
 50%          36.000000
 75%          60.000000
 max          60.000000
 Name: term, dtype: float64, 0    36
 1    60
 2    36
 3    36
 4    60
 Name: term, dtype: int64)

- Make a column named `loan_status_is_great`. It should contain the integer 1 if `loan_status` is "Current" or "Fully Paid." Else it should contain the integer 0.

In [0]:
df['loan_status'].value_counts(dropna=False)

Current               123768
Fully Paid              3438
Late (31-120 days)       509
In Grace Period          441
Late (16-30 days)        223
Charged Off               33
Name: loan_status, dtype: int64

In [0]:
#Create loan_status_is_great from the contains boolean mask
df['loan_status_is_great'] = df['loan_status'].str.contains('Current|Fully Paid', regex=True)

In [0]:
#confirm success
df['loan_status'].head(), df['loan_status_is_great'].head()

(0    Current
 1    Current
 2    Current
 3    Current
 4    Current
 Name: loan_status, dtype: object, 0    True
 1    True
 2    True
 3    True
 4    True
 Name: loan_status_is_great, dtype: bool)

- Make `last_pymnt_d_month` and `last_pymnt_d_year` columns.

In [0]:
#take a look at what we are working with, a few NaN's we can just carry over
df['last_pymnt_d'].value_counts(dropna=False)

2019-02-01    123797
2019-01-01      2464
2018-12-01      1074
2018-11-01       681
2018-10-01       234
NaT              162
Name: last_pymnt_d, dtype: int64

In [0]:
#create last payment month and year
df['last_pymnt_d_month'] = df['last_pymnt_d'].dt.month
df['last_pymnt_d_year'] = df['last_pymnt_d'].dt.year

In [0]:
#inspect for success
df['last_pymnt_d_month'].head(), df['last_pymnt_d_year'].head()

(0    2.0
 1    2.0
 2    2.0
 3    2.0
 4    2.0
 Name: last_pymnt_d_month, dtype: float64, 0    2019.0
 1    2019.0
 2    2019.0
 3    2019.0
 4    2019.0
 Name: last_pymnt_d_year, dtype: float64)

In [0]:
#NaN's carried over as expected
df['last_pymnt_d_month'].value_counts(dropna=False), df['last_pymnt_d_year'].value_counts(dropna=False)

( 2.0     123797
  1.0       2464
  12.0      1074
  11.0       681
  10.0       234
 NaN         162
 Name: last_pymnt_d_month, dtype: int64,  2019.0    126261
  2018.0      1989
 NaN           162
 Name: last_pymnt_d_year, dtype: int64)

# STRETCH OPTIONS

You can do more with the LendingClub or Instacart datasets.

##LendingClub options:
- There's one other column in the dataframe with percent signs. Remove them and convert to floats. You'll need to handle missing values.

In [0]:
#looking for the column with '%', find revol_util
df.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,sec_app_mths_since_last_major_derog,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term,emp_title_manager,issue_month,issue_year,dif_earliest_cr_to_issue,loan_status_is_great,last_pymnt_d_month,last_pymnt_d_year
0,NaN,NaN,2500,2500,2500.0,36,13.56,84.92,C,C1,Chef,10+ years,RENT,55000.0,Not Verified,2018-12-01,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,109xx,NY,18.24,0,2001-04-01,1,NaN,45.0,9,1,4341,10.3%,34,w,2386.02,2386.02,167.02,167.02,113.98,53.04,0.0,0.0,0.0,2019-02-01,84.92,2019-03-01,2019-02-01,0,NaN,1,Individual,NaN,NaN,NaN,0,0,16901,2,2,1,2,2.0,12560,69.0,2,7,2137,28.0,42000,1,11,2,9,1878.0,34360.0,5.9,0,0,140.0,212,1,1,0,1.0,NaN,2.0,NaN,0,2,5,3,3,16,7,18,5,9,0.0,0,0,3,100.0,0.0,1,0,60124,16901,36500,18124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,False,12.0,2018.0,6453,True,2.0,2019.0
1,NaN,NaN,30000,30000,30000.0,60,18.94,777.23,D,D2,Postmaster,10+ years,MORTGAGE,90000.0,Source Verified,2018-12-01,Current,n,NaN,NaN,debt_consolidation,Debt consolidation,713xx,LA,26.52,0,1987-06-01,0,71.0,75.0,13,1,12315,24.2%,44,w,29387.75,29387.75,1507.11,1507.11,612.25,894.86,0.0,0.0,0.0,2019-02-01,777.23,2019-03-01,2019-02-01,0,NaN,1,Individual,NaN,NaN,NaN,0,1208,321915,4,4,2,3,3.0,87153,88.0,4,5,998,57.0,50800,2,15,2,10,24763.0,13761.0,8.3,0,0,163.0,378,4,3,3,4.0,NaN,4.0,NaN,0,2,4,4,9,27,8,14,4,13,0.0,0,0,6,95.0,0.0,1,0,372872,99468,15000,94072,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN,False,12.0,2018.0,11506,True,2.0,2019.0
2,NaN,NaN,5000,5000,5000.0,36,17.97,180.69,D,D1,Administrative,6 years,MORTGAGE,59280.0,Source Verified,2018-12-01,Current,n,NaN,NaN,debt_conso

In [0]:
#strip and recast
df['revol_util'] = df['revol_util'].str.strip('%').astype(float)

In [0]:
#confirm
df['revol_util'].value_counts(dropna=False).head()

0.0     1132
43.0     225
49.0     222
39.1     220
32.0     217
Name: revol_util, dtype: int64

- Modify the `emp_title` column to replace titles with 'Other' if the title is not in the top 20. 

I want to do a variant on this. Consolidate different titles (such as all managers) and then apply Other to anything below the top 25

- create new column `trunc_emp_title` so I don't lose any data
- find titles to consolidate on
- consolidate those titles
- get list of top 25 titles
- consolidate all titles not in top 25 to `Other`

In [0]:
#step one, new column
df['trunc_emp_title'] = df['emp_title']

In [29]:
#step two, find titles to consolidate on
df['emp_title'].value_counts()

# Nurse - Nurse, Registered Nurse, Dental Hygienist, Dental Assistant, Nursing Assistant, Phlebotomist, Medical Technologist, Lpn, Rn, Nurse Practitioner, Respiratory Therapist, Pharmacy Technician, Physical Therapist, Physician Assistant, Medical Technologist, Lvn, Certified Nursing Assistant
# Doctor - Doctor, Physician, Surgeon, Pharmacist, Dentist
# Emergency Services - Police, Firefighter, EMT, Paramedic, Emergency Medical Technician, Sheriff, Deputy, Detective, Dispatcher, Officer
# Supervisor -  Supervisor, Maintenance Supervisor, Production Supervisor, Operations Supervisor, Warehouse Supervisor, Shift Supervisor, Field Supervisor, Customer Service Supervisor, Security Supervisor, Sales Supervisor, Area Supervisor, Shop Supervisor, Site Supervisor, Accounting Supervisor, Claims Supervisor, Transportation Supervisor, Shipping Supervisor, Billing Supervisor, Office Supervisor, Clinical Supervisor, Team Leader, Team Lead
# Manager - Manager, Gm, 'Office Manager, General Manager, Project Manager, Operations Manager, Sales Manager, Store Manager, Assistant Manager, Account Manager, Branch Manager, Program Manager, Service Manager, District Manager, Case Manager, Production Manager, Property Manager, Finance Manager, It Manager, Accounting Manager, Senior Manager, Business Manager, Warehouse Manager, Marketing Manager, Assistant Store Manager, Department Manager, Regional Manager, Area Manager, Hr Manager, Regional Sales Manager, Plant Manager, Product Manager, Senior Project Manager, Business Development Manager, Territory Manager, Customer Service Manager, Parts Manager, Quality Manager, Maintenance Manager, Human Resources Manager, Relationship Manager, Purchasing Manager, Practice Manager, Construction Manager, Logistics Manager, It Project Manager, Senior Account Manager, Engineering Manager, Nurse Manager, Facilities Manager, Assistant General Manager, Operation Manager, Facility Manager, Sr. Project Manager, Site Manager, Rn Case Manager, Fleet Manager, Kitchen Manager, General Sales Manager, Restaurant Manager, Billing Manager, Shift Manager, District Sales Manager, Assistant Branch Manager, Shipping Manager, Payroll Manager, Transportation Manager, Safety Manager, Compliance Manager, Division Manager, Asst Manager, Credit Manager, Community Manager, Bar Manager, Clinical Manager, Retail Manager, Territory Sales Manager, Administrative Manager, Pharmacy Manager, Assistant Project Manager, Shop Manager, Inventory Manager, Business Office Manager, National Account Manager, Assistant Property Manager, Deli Manager, Market Manager, Sr. Manager, Claims Manager, Human Resource Manager, Portfolio Manager, Produce Manager, Qa Manager, Sr Project Manager, Senior Program Manager, National Sales Manager, Support Manager, Key Account Manager, Dept Manager, Area Sales Manager, Front Office Manager, Customer Success Manager, Security Manager, Field Manager, Grocery Manager, Terminal Manager, Food Service Manager, Financial Manager, Asset Manager, Technical Manager, Manufacturing Manager, Audit Manager, Team Manager, Meat Manager, Center Manager, Receiving Manager, Asst. Manager, Accounts Payable Manager, Unit Manager, Risk Manager, Tax Manager, Senior Product Manager, Training Manager, Sr Manager, Technical Project Manager, Front Desk Manager, Client Manager, Lab Manager, Registered Nurse Case Manager, Materials Manager, Sales Account Manager, Internet Manager, Catering Manager, Associate Manager, Client Service Manager, Retail Store Manager, Contract Manager, Care Manager, Quality Control Manager, Quality Assurance Manager, Assistant Service Manager, Data Manager, Internet Sales Manager, Recruiting Manager, Clinic Manager, Food And Beverage Manager, Category Manager
# Director -  Director, Director Of Operations, Executive Director, Managing Director, Assistant Director, Director Of Sales, Sales Director, Program Director, Marketing Director, Associate Director, It Director, Finance Director, Senior Director, Director Of Nursing, Director Of Marketing, Creative Director, Director Of Finance, Director Of Human Resources, Regional Director, Hr Director, Operations Director, Director Of Business Development, Deputy Director, Director Of Technology, Director Of Development, Project Director, Art Director, Director Of It, Human Resources Director, Store Director, Food Service Director, Clinical Director, Director Of Maintenance, Service Director, Funeral Director, Director Of Clinical Services, Regional Sales Director, Safety Director, Director Of Engineering, Technical Director, Sr. Director
# Executive - President, CEO, Owner, Vice President, Cfo, Vp, Coo, Partner, Chief Executive Officer, Chief Operating Officer
# Engineer -  Engineer, Software Engineer, Systems Engineer, Senior Software Engineer, Project Engineer, Network Engineer, Electrical Engineer, Quality Engineer, Mechanical Engineer, Field Engineer, Engineering Manager, Sales Engineer, Senior Engineer, Locomotive Engineer, System Engineer, Civil Engineer, Chief Engineer, Engineering Technician, Manufacturing Engineer, Field Service Engineer, Process Engineer, Design Engineer, Sr. Software Engineer, Engineering, Building Engineer, Stationary Engineer, Operating Engineer, Senior Systems Engineer, It Engineer, Principal Engineer, Security Engineer, Maintenance Engineer, Application Engineer, Director Of Engineering, Senior Project Engineer, Product Engineer, Principal Software Engineer, Senior Network Engineer
# Legal - Attorney, Paralegal, Legal Assistant, Legal Secretary
# Educator - Teacher, Superintendent, Principal, Instructor, School Counselor, Associate Professor, Professor, Coach, Special Education Teacher, Assistant Principal, Assistant Professor
# Financial Services - Accountant, Analyst, Cna, Controller, Bookkeeper, Loan Officer, Financial Advisor, Financial Analyst, Staff Accountant, Underwriter, Senior Accountant, Banker, Personal Banker, Loan Processor
# Driver - Driver, Truck Driver, Cdl Driver, Delivery Driver, Cab Driver, Taxi Driver


Unknown                                   20947
Teacher                                    2557
Manager                                    2395
Registered Nurse                           1418
Driver                                     1258
Supervisor                                 1160
Truck Driver                                920
Rn                                          834
Office Manager                              805
Sales                                       803
General Manager                             791
Project Manager                             720
Owner                                       625
Director                                    523
Operations Manager                          518
Sales Manager                               500
Police Officer                              440
Nurse                                       425
Technician                                  420
Engineer                                    412
Administrative Assistant                

In [0]:
#function to take my note-taking above and convert to a dictionary to use with .replace
def write_dict_from(a_string, a_word):
  a_dict = {}
  a_list = a_string.split(', ')
  for word in a_list:
    a_dict[word] = a_word
  return a_dict

In [0]:
manager_string =  'Gm, Office Manager, General Manager, Project Manager, Operations Manager, Sales Manager, Store Manager, Assistant Manager, Account Manager, Branch Manager, Program Manager, Service Manager, District Manager, Case Manager, Production Manager, Property Manager, Finance Manager, It Manager, Accounting Manager, Senior Manager, Business Manager, Warehouse Manager, Marketing Manager, Assistant Store Manager, Department Manager, Regional Manager, Area Manager, Hr Manager, Regional Sales Manager, Plant Manager, Product Manager, Senior Project Manager, Business Development Manager, Territory Manager, Customer Service Manager, Parts Manager, Quality Manager, Maintenance Manager, Human Resources Manager, Relationship Manager, Purchasing Manager, Practice Manager, Construction Manager, Logistics Manager, It Project Manager, Senior Account Manager, Engineering Manager, Nurse Manager, Facilities Manager, Assistant General Manager, Operation Manager, Facility Manager, Sr. Project Manager, Site Manager, Rn Case Manager, Fleet Manager, Kitchen Manager, General Sales Manager, Restaurant Manager, Billing Manager, Shift Manager, District Sales Manager, Assistant Branch Manager, Shipping Manager, Payroll Manager, Transportation Manager, Safety Manager, Compliance Manager, Division Manager, Asst Manager, Credit Manager, Community Manager, Bar Manager, Clinical Manager, Retail Manager, Territory Sales Manager, Administrative Manager, Pharmacy Manager, Assistant Project Manager, Shop Manager, Inventory Manager, Business Office Manager, National Account Manager, Assistant Property Manager, Deli Manager, Market Manager, Sr. Manager, Claims Manager, Human Resource Manager, Portfolio Manager, Produce Manager, Qa Manager, Sr Project Manager, Senior Program Manager, National Sales Manager, Support Manager, Key Account Manager, Dept Manager, Area Sales Manager, Front Office Manager, Customer Success Manager, Security Manager, Field Manager, Grocery Manager, Terminal Manager, Food Service Manager, Financial Manager, Asset Manager, Technical Manager, Manufacturing Manager, Audit Manager, Team Manager, Meat Manager, Center Manager, Receiving Manager, Asst. Manager, Accounts Payable Manager, Unit Manager, Risk Manager, Tax Manager, Senior Product Manager, Training Manager, Sr Manager, Technical Project Manager, Front Desk Manager, Client Manager, Lab Manager, Registered Nurse Case Manager, Materials Manager, Sales Account Manager, Internet Manager, Catering Manager, Associate Manager, Client Service Manager, Retail Store Manager, Contract Manager, Care Manager, Quality Control Manager, Quality Assurance Manager, Assistant Service Manager, Data Manager, Internet Sales Manager, Recruiting Manager, Clinic Manager, Food And Beverage Manager, Category Manager, Asst.Manager, Yard Manager/Safety Inspector, Unclaimed Property Claims Manager, International Claims Manager, Management'
nurse_string = 'Registered Nurse, Dental Hygienist, Dental Assistant, Nursing Supervisor, Rn Supervisor, Nursing Assistant, Phlebotomist, Medical Technologist, Lpn, Rn, Nurse Practitioner, Respiratory Therapist, Pharmacy Technician, Physical Therapist, Physician Assistant, Medical Technologist, Lvn, Certified Nursing Assistant'
doctor_string = 'Physician, Surgeon, Pharmacist, Dentist'
emergency_string = 'Police, Firefighter, EMT, Paramedic, Emergency Medical Technician, Sheriff, Deputy, Detective, Dispatcher, Officer, Deputy Sheriff, Police Officer'
supervisor_string = 'Maintenance Supervisor, Production Supervisor, Operations Supervisor, Warehouse Supervisor, Shift Supervisor, Field Supervisor, Customer Service Supervisor, Security Supervisor, Sales Supervisor, Area Supervisor, Shop Supervisor, Site Supervisor, Accounting Supervisor, Claims Supervisor, Transportation Supervisor, Shipping Supervisor, Billing Supervisor, Office Supervisor, Clinical Supervisor, Team Leader, Team Lead'
director_string = 'Director, Director Of Operations, Executive Director, Managing Director, Assistant Director, Director Of Sales, Sales Director, Program Director, Marketing Director, Associate Director, It Director, Finance Director, Senior Director, Director Of Nursing, Director Of Marketing, Creative Director, Director Of Finance, Director Of Human Resources, Regional Director, Hr Director, Operations Director, Director Of Business Development, Deputy Director, Director Of Technology, Director Of Development, Project Director, Art Director, Director Of It, Human Resources Director, Store Director, Food Service Director, Clinical Director, Director Of Maintenance, Service Director, Funeral Director, Director Of Clinical Services, Regional Sales Director, Safety Director, Director Of Engineering, Technical Director, Sr. Director'
executive_string = 'President, CEO, Owner, Vice President, Cfo, Vp, Coo, Partner, Chief Executive Officer, Chief Operating Officer'
engineer_string = 'Software Engineer, Systems Engineer, Senior Software Engineer, Project Engineer, Network Engineer, Electrical Engineer, Quality Engineer, Mechanical Engineer, Field Engineer, Engineering Manager, Sales Engineer, Senior Engineer, Locomotive Engineer, System Engineer, Civil Engineer, Chief Engineer, Engineering Technician, Manufacturing Engineer, Field Service Engineer, Process Engineer, Design Engineer, Sr. Software Engineer, Engineering, Building Engineer, Stationary Engineer, Operating Engineer, Senior Systems Engineer, It Engineer, Principal Engineer, Security Engineer, Maintenance Engineer, Application Engineer, Director Of Engineering, Senior Project Engineer, Product Engineer, Principal Software Engineer, Senior Network Engineer'
legal_string = 'Attorney, Paralegal, Legal Assistant, Legal Secretary'
educator_string = 'Teacher, Superintendent, Principal, Instructor, School Counselor, Associate Professor, Professor, Coach, Special Education Teacher, Assistant Principal, Assistant Professor'
financial_string = 'Accountant, Analyst, Cna, Controller, Bookkeeper, Loan Officer, Financial Advisor, Financial Analyst, Staff Accountant, Underwriter, Senior Accountant, Banker, Personal Banker, Loan Processor'
driver_string = 'Truck Driver, Cdl Driver, Delivery Driver, Cab Driver, Taxi Driver, Bus Driver, Bus Operator, School Bus Driver'

list_of_strings = [nurse_string, doctor_string, emergency_string, director_string, executive_string, engineer_string, legal_string, educator_string, financial_string, manager_string, supervisor_string, driver_string]
dict_values = ['Nurse', 'Doctor', 'Emergency Services', 'Director', 'Executive', 'Engineer', 'Legal Services', 'Educator', 'Financial Services', 'Manager', 'Supervisor', 'Driver']

In [0]:
#step three, consolidate those titles
for i in range(len(dict_values)):
  df['trunc_emp_title'] = df['trunc_emp_title'].str.strip().replace(write_dict_from(list_of_strings[i],dict_values[i]))

In [44]:
#step four, get list of top 25 titles
top_25 = df['trunc_emp_title'].value_counts().head(25).index.tolist()
top_25

['Unknown',
 'Manager',
 'Nurse',
 'Educator',
 'Driver',
 'Director',
 'Supervisor',
 'Financial Services',
 'Engineer',
 'Executive',
 'Emergency Services',
 'Sales',
 'Legal Services',
 'Technician',
 'Administrative Assistant',
 'Mechanic',
 'Server',
 'Electrician',
 'Foreman',
 'Doctor',
 'Machine Operator',
 'Customer Service',
 'Executive Assistant',
 'Operator',
 'Maintenance']

In [0]:
#step 5, consolidate all titles not in top 25 to Other
#create a logic mask and then assign all titles not in top_25 to 'Other'
mask = np.logical_not(df['trunc_emp_title'].isin(top_25))
df.loc[mask,'trunc_emp_title'] = 'Other'

In [47]:
#validate
df['trunc_emp_title'].value_counts()

Other                       69399
Unknown                     20947
Manager                     12379
Nurse                        3930
Educator                     3632
Driver                       2583
Director                     1975
Supervisor                   1953
Financial Services           1903
Engineer                     1690
Executive                    1594
Emergency Services           1061
Sales                         803
Legal Services                716
Technician                    420
Administrative Assistant      409
Mechanic                      391
Server                        357
Electrician                   302
Foreman                       302
Doctor                        300
Machine Operator              284
Customer Service              283
Executive Assistant           278
Operator                      267
Maintenance                   254
Name: trunc_emp_title, dtype: int64

- Take initiatve and work on your own ideas!

##Instacart options:
- Read [Instacart Market Basket Analysis, Winner's Interview: 2nd place, Kazuki Onodera](http://blog.kaggle.com/2017/09/21/instacart-market-basket-analysis-winners-interview-2nd-place-kazuki-onodera/), especially the **Feature Engineering** section. (Can you choose one feature from his bulleted lists, and try to engineer it with pandas code?)
- Read and replicate parts of [Simple Exploration Notebook - Instacart](https://www.kaggle.com/sudalairajkumar/simple-exploration-notebook-instacart). (It's the Python Notebook with the most upvotes for this Kaggle competition.)
- Take initiative and work on your own ideas!

You can uncomment and run the cells below to re-download and extract the Instacart data

In [0]:
# !wget https://s3.amazonaws.com/instacart-datasets/instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# !tar --gunzip --extract --verbose --file=instacart_online_grocery_shopping_2017_05_01.tar.gz

In [0]:
# %cd instacart_2017_05_01